## Startup

First, install `particula` from PyPI using `!pip install -U particula` or `%pip install -U particula`. In this, the `!` or `%` are so-called magic signs prefixing "magic functions". In this case, the magic function is `%pip` --- which is one way to install Python packages from the PyPI (the Python Package Index). The `-U` is to instruct `pip` to update particula to the latest version.

In [1]:
%pip install -U particula

After installing `particula`, we can import it into our workspace with `import particula`. Let's check the version with `.__version__` and let's call `help` on the overall package to see what we have. 

In [2]:
import particula

In [3]:
particula.__version__

'0.0.2'

In [4]:
help(particula)

Help on package particula:

NAME
    particula - PARTICULA!

PACKAGE CONTENTS
    aerosol_dynamics (package)

VERSION
    0.0.2

FILE
    /usr/local/lib/python3.7/dist-packages/particula/__init__.py




The above two code cells show that we have the prerelease version `0.0.2` (our latest as of writing this). We still are in the process of writing more thorough documentation, and so a help call on `particula` yields a shout --- maybe a shout for help? `PARTICULA!` However, we get a hint about what else is inside this package, `aerosol_dynamics` package/module. We will check this now.

## A peek into `aerosol_dynamics`

We can import `aerosol_dynamics` either via 

```python
from particula import aerosol_dynamics
```

or

```python
import particula.aerosol_dynamics
```

to check it out. We will call help again to see what we have. 

In [5]:
from particula import aerosol_dynamics

In [6]:
help(aerosol_dynamics)

Help on package particula.aerosol_dynamics in particula:

NAME
    particula.aerosol_dynamics - This is the aerosol_dynamics module.

DESCRIPTION
    It can do the following:

PACKAGE CONTENTS
    environment
    environment_test
    parcel
    parcel_test
    particle
    particle_test
    physical_parameters

DATA
    u = <pint.registry.UnitRegistry object>

FILE
    /usr/local/lib/python3.7/dist-packages/particula/aerosol_dynamics/__init__.py




Now! We have some package contents that look promising. We will only check out `particle` in this case. Again, import it and call help on it.

In [7]:
from particula.aerosol_dynamics import particle

In [8]:
help(particle)

Help on module particula.aerosol_dynamics.particle in particula.aerosol_dynamics:

NAME
    particula.aerosol_dynamics.particle - Class for creating particles.

CLASSES
    builtins.object
        Particle
    
    class Particle(builtins.object)
     |  Particle(name: str, radius, density, charge)
     |  
     |  Class for creating particles:
     |  Framework for particle--particle and gas--particle interactions.
     |  
     |  Attributes:
     |      name    (str)   [no units]
     |      radius  (float) [m]
     |      density (float) [kg/m**3]
     |      charge  (int)   [dimensionless]
     |      mass    (float) [kg]
     |  
     |  Methods defined here:
     |  
     |  __init__(self, name: str, radius, density, charge)
     |      Constructs the particle object.
     |      
     |      Parameters:
     |          name    (str)   [no units]
     |          radius  (float) [m]
     |          density (float) [kg/m**3]
     |          charge  (int)   [dimensionless]
     |  

We see there is more documentation now. The `help` call lists a "class" called "Particle" and all its available "methods" (read: functions) alphabetically. For now, you can think of a class as a super-function (it is more than that). Note also, that classes follow `CamelCase` (hence, `Particle`) whereas methods/functions follow snake_case (hence, `reduced_mass`). We make a lot of use of unit registry (called `u`) to ensure all our calculations have proper units, even if dimensionless. Without further ado, let's look at an example.

## An example... 

... to calculate the dimensionless coagulation rate. Let's define to particles, `particla_a` and `particle_b`.

In [9]:
particle_a = particle.Particle(name='a', radius=1e-9, density=1e3, charge=0)

In [10]:
particle_b = particle.Particle(name='b', radius=2e-9, density=1e3, charge=0)

Then, if we call the `radius` method of particle_a (as suffix), we get its radius printed with a unit. Alternatively, we can call the `radius` method from the class itself, not prefixed to a certain particle (same result).

In [11]:
particle_a.radius()

1e-09 meter

In [12]:
particle.Particle.radius(particle_a)

1e-09 meter

Same for the `mass` and `charge` methods. Note `charge` here is the number of charges. 

In [13]:
particle_b.mass()

3.3510321638291136e-23 kilogram

In [14]:
particle_a.charge()

0 dimensionless

We should also define the environment we are in. We will get that module and invoke with 300 K and $10^5$ Pa. 

In [15]:
from particula.aerosol_dynamics import environment

In [16]:
std_environment = environment.Environment(temperature=300, pressure=1e5)

Now, we can calculate the Knudsen number of particle "a" in this "standard" environment. And the slip correction factor. And the friction factor. 

In [17]:
particle_a.knudsen_number(std_environment)

67.8918099704083 dimensionless

In [18]:
particle_a.slip_correction_factor(std_environment)

113.060274442495 dimensionless

In [19]:
particle_a.friction_factor(std_environment)

3.0775355689697488e-15 kilogram/second

Now, let's calculate the reduced mass of two particles, "a" and "b". Again, there are two ways to do this: by suffixing the method to a particle or by calling the method from the class --- they yield the same result. 

In [20]:
particle_a.reduced_mass(particle_b)

3.7233690709212374e-24 kilogram

In [21]:
particle.Particle.reduced_mass(particle_a, particle_b)

3.7233690709212374e-24 kilogram

Like the reduced mass, we can also look at the reduced friction. 

In [22]:
particle_a.reduced_friction_factor(particle_b, std_environment)

2.4595460300833962e-15 kilogram/second

Finally, we can calculate the coagulation rate coefficient between particles "a" and "b" in this environment. Note that we could have called this directly after defining "a" and "b".

In [23]:
particle.Particle.dimensionless_coagulation_kernel_hard_sphere(
    particle_a, particle_b, std_environment
)

83.4628646051075 dimensionless

In [24]:
particle_a.dimensionless_coagulation_kernel_hard_sphere(
    particle_b, std_environment
)

83.4628646051075 dimensionless